In [1]:
#Imports
import pandas as pd
import numpy as np 
import nilearn 
import nibabel as nib

from nilearn.maskers import NiftiMasker
from nilearn import image

In [6]:
# -- Load SIGMA atlas and infos --

template_folder = '/groupshare/traaffneu/preclinimg/templates/'

atlas_img = nib.load(template_folder+'SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Brain_Atlases/SIGMA_Anatomical_Atlas/SIGMA_Anatomical_Brain_Atlas.nii')
atlas_data = atlas_img.get_fdata()

atlas_labels = (template_folder+'SIGMA_Wistar_Rat_Brain_TemplatesAndAtlases_Version1.1/SIGMA_Rat_Brain_Atlases/SIGMA_Anatomical_Atlas/SIGMA_Anatomical_Brain_Atlas_Labels.txt')
label_info = np.genfromtxt(atlas_labels, skip_header=10, delimiter='\t', dtype=str)


z_scores = nib.load('/project/4180000.19/multirat_stim/rabies_test/first_level_analysis/z_score/z_score_sub-0200100_ses-1.nii.gz')
subject_path = '/project/4180000.19/multirat_stim/bids/sub-0200100/ses-1/func/sub-0200100_ses-1_run-1_bold.nii.gz'



In [7]:
#Find index of the ROI 

label_to_find = "Primary Somatosensory Cortex Forelimb-L"      #to be defined in metadata 


for row in label_info:                                   
    if row[-1].strip('"') == label_to_find:         #access the last column of each row, remove quotes, compare
        label_idx = int(row[0])                     #if match, assign the index found at row0 to label_index
        print(label_idx)
        break
else:
    raise ValueError("Label '{}' not found in the label file.".format(label_to_find))


551


In [8]:
from nilearn import image

subj_img = nib.load(subject_path)
print(type(subj_img))

subj_img_3d = image.index_img(subj_img, 0)

print(subj_img_3d.shape)
print(type(subj_img_3d))

#plotting.plot_stat_map(subj_img_3d, bg_img = template_path)


<class 'nibabel.nifti1.Nifti1Image'>
(64, 12, 64)
<class 'nibabel.nifti1.Nifti1Image'>


In [12]:
#Extract data from ROI

roi_mask = (nilearn.image.get_data(atlas_img) == label_idx).astype(bool)  # select ROI based on the index
mask_img = nilearn.image.new_img_like(atlas_img, roi_mask)                # create an mask image out of the roi selected

masker = NiftiMasker(mask_img=mask_img)                                   #apply a mask to extract time-series from subj_img
     
# roi_timeseries = masker.fit_transform(subj_img)
# roi_residuals = masker.fit_transform(fmri_glm.residuals[0])    
# roi_effect_size = masker.fit_transform(statistical_map['effect_size'])
# roi_effect_variance = masker.fit_transform(statistical_map['effect_variance'])
# roi_predicted_timeseries = masker.fit_transform(fmri_glm.predicted[0])

report = masker.generate_report()
report

/home/traaffneu/margal/.conda/envs/MultiRatStim/lib/python3.9/site-packages/nilearn/reporting/html_report.py:191: UserWarning: This object has not been fitted yet ! Make sure to run `fit` before inspecting reports.
  warnings.warn('This object has not been fitted yet ! '


Parameter,Value
